# Process parish maps data

I started with an export from the SLV catalogue containing metadata describing [8,804 parish maps](https://find.slv.vic.gov.au/discovery/search?query=series,exact,Parish%20maps%20of%20Victoria&vid=61SLV_INST:SLV&offset=0). The SLV cataloguers have geocoded many of these maps using the [Composite Gazetteer of Australia](https://placenames.fsdf.org.au), which provides coordinates for Victorian parishes and boroughs. These coordinates give us a point which should be roughly at the centre of the map. Using these points, we can visualise the locations and distribution of the maps, but how much area do they cover? To answer that we need a bounding box that describes the coordinates of each corner of the map. We could use something like AllMaps or MapWarper to georeference each individual map, but that's going to take a while! As a quick and dirty alternative, I wondered if it was possible to generate approximate bounding boxes from the available metadata. This notebook documents that process.

## The metadata

There are three pieces of metadata we need to construct bounding boxes:

- the latitude and longitude of the centre point
- the size of the physical map
- the scale of the map (ie how the size of the map relates to real world)

The coordinates and scale can be included in a couple of different places in the map's MARC record. The [`034`](https://www.loc.gov/marc/bibliographic/bd034.html) field is specifically for 'Coded Cartographic Mathematical Data'. The relevant subfields are:

- `$a`: category of scale
- `$b`: constant ratio linear horizontal scale (this is the most likely type of scale)
- `$d`: westernmost longitude
- `$e`: easternmost longitude
- `$f`: northernmost latitude
- `$g`: southernmost latitude

If the coordinates describe a point rather than a bounding box, then `$d` and `$e` will be the same, and `$f` and `$g` will be the same.

String representations of coordinates and scale can be found in the `255` field. The relevant subfields are:

- `$a`: statement of scale, eg `Scale [ca. 1:90,000].`
- `$c`: statement of coordinates, eg `(E 142°18'/S 37°33')`

The size of the map is recorded in [`300`](https://www.loc.gov/marc/bibliographic/bd300.html) (physical description) field under the `$c` (dimensions) subfield. For example: `on sheet 40 x 51 cm `.

## The method

I started with an existing dataset downloaded from the catalogue by SLV staff. This dataset included the scale and coordinate information in the `034` field, and the coordinate string in `255$c`. At first I didn't realise that the `034` held geo data, so I separately downloaded the scale information from `255:$a` in each item's MARC record (d'oh). If the maps were digitised, I also wanted their image identifiers so I could access them through IIIF. The image id from the `956$e` field of the MARC record can be used to construct an IIIF manifest url, so I extracted them as well.

Once I had the catalogue data, I had to make sure everything was in a format I could work with. The coordinates in the MARC records are recorded as degrees/minutes/seconds, so I had to convert them to decimal values. The scale factor needed to be an integer, and I needed to extract the height and width as integers from the dimensions field.

I used [lat_lon_parser](https://pypi.org/project/lat-lon-parser/) to convert the coordinates to decimal, but needed a bit of regex string manipulation to get the values into a format that could be parsed. Regex also came to the rescue in getting the map dimensions. All the details are in the code below.

### Creating bounding boxes

After some searching I found [this StackOverflow comment](https://stackoverflow.com/a/76910048) that described how to create a bounding box from a point, distance, and bearing. The point I already had, but the distance and bearing had to be calculated. Trigonometry to the rescue!

<img src="https://updates.timsherratt.org/uploads/2025/parish-maps-box-trig.png" width="600" height="334" alt="">

The distance from the point at the centre of the box to one of its corners is the hypotenuse of a right-angled triangle whose sides are equal to half the width and half the height of the map, and thanks to Pythagorus we know:

distance<sup>2</sup> = √(height/2)<sup>2</sup>) + (width/2)<sup>2</sup>

Once I had the distance in cm, I converted to inches, then multiplied by the scale factor, and finally converted the inches to miles. (It now occurs to me that there's no need to convert to imperial measurements, but it doesn't make any difference either way.)

The bearing that points to the corner of the box is the angle inside the same right-angled triangle, so can be calculated using tan((width/2)/(height/2)).

With the point, distance, and bearing I could use [geopy](https://github.com/geopy/geopy) to calculate the bounding box!

## Limitations

Of course, this method is very rough and has a number of major limitations, in particular:

- only about 38% of the maps have point coordinates (see below for a summary of the available metadata)
- the point values don't necessarily locate the centre of the map
- not all the maps are oriented towards north
- sometimes a parish includes multiple maps
- the size of the margin around the map will affect the accuracy of the bounding box

But despite these problems the results seem pretty good. To test this I overlaid digitised maps on a modern basemap using the bounding boxes. Here's an example.

<img src="https://updates.timsherratt.org/uploads/2025/parish-maps-overlay.png" width="600" height="471" alt="">

You can see the map is slightly offset (presumably due to the first problem listed above). But the size seems about right. Certainly good enough to use the bounding boxes in some exploratory analyses!

## Results

I've saved my results as a new dataset, and started playing around with a couple of ways of visualising the results – see the [parish maps browser](parish_maps_browser.ipynb) and a [visualisation of all the bounding boxes](parish_maps_visualise_bounds.ipynb).

In [2]:
import math
import re
import time
import unicodedata
import warnings

import geopy
import ipywidgets as widgets
import pandas as pd
import requests
from geopy.distance import geodesic
from ipyleaflet import ImageOverlay, Map, WidgetControl
from lat_lon_parser import parse, to_dec_deg
from requests_cache import CachedSession
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

sess = CachedSession(timeout=60)
tqdm.pandas()

In [3]:
# Import the original dataset
df = pd.read_csv("parish_maps.csv")[
    [
        "MMS ID - 001",
        "Title - 245$a",
        "Coordinates - 255$c",
        "034$d",
        "034$f",
        "Publisher details - 260",
        "Description - 300 ",
    ]
]

In [4]:
# Rename columns for convenience
df.columns = [
    "alma_id",
    "title",
    "coordinates",
    "east",
    "south",
    "publisher",
    "description",
]

In [5]:
df

,alma_id,title,coordinates,east,south,publisher,description
0,9921177713607636,"Paaratte, County of Heytesbury",(E 142°59'/S 38°33').,E1425900,S0383300,"Melbourne :Dept. of Lands and Survey,1882.",1 map ;on sheet 58 x 42 cm.
1,9921177763607636,"Powlett, County of Gladstone",(E 143°51'/S 36°25').,E1435100,S0362500,"Melbourne :Dept. of Lands and Survey,1883.",1 map ;on sheet 33 x 69 cm.
2,9921177843607636,"Patho, County of Gunbower",(E 144°22'/S 35°59').,E1442200,S0353900,"Melbourne :H. J. Green, Govt. Printer,1926.",1 map ;on sheet 54 x 81 cm.
3,9921177883607636,[Map of Parish of Stewarton],(E 145°50'/S 36°25' ).,E1455000,S0362500,"[Melbourne? :Dept. of Lands and Survey?,188-?].",1 map ;on sheet 30 x 41 cm.
4,9921178363607636,Plan of a road from Stanley to Hillsborough an...,NaN,NaN,NaN,"Melbourne :Dept. of Lands and Survey,1867.",1 map ;on sheet 56 x 38 cm.
...,...,...,...,...,...,...,...
8799,9920315583607636,"Parish of Bundalong, County of Moira",NaN,NaN,NaN,1982.,1 map.
8800,9920315593607636,"Parish of Bundalong, County of Moira",NaN,NaN,NaN,1982.,1 map.
8801,9920315603607636,"Parish of Bundalong, County of Moira, Schedule",NaN,NaN,NaN,1982.,1 map.
8802,9920315613607636,"Parish of Bungal, County of Grant",NaN,NaN,NaN,1982.,1 map.


How many have coordinates?

In [27]:
df.loc[(df["coordinates"].notnull()) & (df["latitude"].notnull())].shape[0]

3313

In [36]:
print(f"{(df.loc[(df['coordinates'].notnull())].shape[0] / df.shape[0]):.02%} of maps have point coordinates")

37.74% of maps have point coordinates


In [6]:
def get_image_id(alma_id):
    """
    Get the IE image identifier from the MARC record.
    These ids are used to construct IIIF manifest urls.
    """
    marc = get_marc_record(alma_id)
    try:
        image_id = re.search(r"\$e(IE\d+)", marc).group(1)
    except AttributeError:
        # print(alma_id)
        image_id = ""
    return image_id


def get_iiif_ids(image_id):
    """
    Extract a list of image @ids from an IIIF manifest
    """
    image_ids = []
    if image_id:
        manifest_url = f"https://rosetta.slv.vic.gov.au/delivery/iiif/presentation/2.1/{image_id}/manifest.json"
        try:
            response = sess.get(manifest_url)
        except requests.exceptions.Timeout:
            print(f"timeout: {manifest_url}")
        else:
            if response.ok:
                try:
                    manifest = response.json()
                except requests.JSONDecodeError:
                    print(manifest_url)
                else:
                    # There can be multiple images in a record
                    # So we loop through the canvases to get each one.
                    for canvas in manifest["sequences"][0]["canvases"]:
                        if canvas["images"][0]["resource"]["format"] == "image/jpeg":
                            image_ids.append(
                                canvas["images"][0]["resource"]["service"]["@id"]
                            )
            else:
                print(f"{response.status_code}: {manifest_url}")
                print(response.headers)
            if not response.from_cache:
                time.sleep(5)
    # print(image_ids)
    return "|".join(image_ids)


def get_marc_record(alma_id):
    """
    Gets a text representation of an item's MARC record.
    """
    response = sess.get(
        f"https://find.slv.vic.gov.au/primaws/rest/pub/sourceRecord?docId=alma{alma_id}&vid=61SLV_INST:SLV"
    )
    return response.text


def get_marc_value(marc, tag, subfield):
    """
    Gets the value of a tag/subfield from a text version of an item's MARC record.
    """
    try:
        tag = re.search(rf"^{tag}\t.+", marc, re.M).group(0)
        subfield = re.search(rf"\${subfield}([^\$]+)", tag).group(1)
    except AttributeError:
        return None
    return subfield.strip(" .,")


def clean_unicode(value):
    n = unicodedata.normalize("NFKD", value)
    return "".join([c for c in n if not unicodedata.combining(c)])


def fix_coords(value):
    """
    Normalise coordinate strings.
    """
    value = clean_unicode(value)
    if value.startswith("E"):
        value = re.sub(r"(E)\s*(\d{3})(\d{2,})[ʹ']?", r"\1 \2°\3'", value)
        value = re.sub(r"(E)\s*(\d{3})(\d{1})°(\d{1,})[ʹ']?", r"\1 \2°\3\4'", value)
        coord_type = "long"
    elif value.startswith("S"):
        value = re.sub(r"(S)\s*(\d{2})(\d{2,})[ʹ']?", r"\1 \2°\3'", value)
        value = re.sub(r"(S)\s*(\d{2})(\d{1})°(\d{1,})[ʹ']?", r"\1 \2°\3\4'", value)
        coord_type = "lat"
    else:
        coord_type = None
    return value, coord_type


def parse_coords_string(point):
    """
    Convert coordinate strings into decimal coordinates.
    """
    coords = {}
    try:
        for coord in point.split("/"):
            value, coord_type = fix_coords(coord.strip("() .,"))
            if coord_type:
                coords[coord_type] = parse(value)
    except (AttributeError, TypeError, ValueError):
        if pd.notnull(point):
            print(point)
            pass
        return None, None
    return coords.get("lat"), coords.get("long")


def clean_coord(match):
    """
    Reformat coordinate values for parsing.
    """
    coord = match.group(2).strip("0")
    if match.group(1) == "S":
        coord = coord.ljust(6, "0")
        coord = coord.rjust(7, "0")
    else:
        coord = coord.ljust(7, "0")

    return f"{match.group(1)}{coord}"


def parse_coord(coord):
    """
    Convert coordinate value from dddmmss to decimal.
    """
    coord = re.sub(r"(E|S)\s*(\d+)", clean_coord, coord)
    try:
        decimal = to_dec_deg(int(coord[1:4]), int(coord[4:6]), int(coord[6:8]))
        if coord.startswith("S"):
            decimal = 0 - decimal
    except ValueError:
        decimal = None
        print(coord)
    return decimal


def get_coords(row):
    """
    Get decimal coordinates from the values in either MARC field 034 or 255.
    """
    if pd.notnull(row["east"]) and pd.notnull(row["south"]):
        longitude = parse_coord(row["east"])
        latitude = parse_coord(row["south"])
    elif pd.notnull(row["coordinates"]):
        latitude, longitude = parse_coords_string(row["coordinates"])
    else:
        latitude = None
        longitude = None
    return pd.Series([latitude, longitude])


def get_scale(alma_id):
    """
    Gets the scale value from the MARC record fields 255 and 034.
    """
    marc = get_marc_record(alma_id)
    scale = get_marc_value(marc, "255", "a")
    factor = get_marc_value(marc, "34", "b")
    if not scale and not factor:
        return pd.Series([None, None])
    elif not factor:
        try:
            # 1 : 10000, 1: 10,000, ca. 10000 etc
            factor = int(
                re.search(r"(?:1\s?:|ca.)\s*([0-9,\s]{3,})", scale)
                .group(1)
                .replace(",", "")
                .replace(" ", "")
            )
        except AttributeError:
            # print(scale)
            factor = None
    else:
        factor = int(factor)
    return pd.Series([scale, factor])


def get_size(description):
    """
    Get the height and width of a map from the MARC description field.
    """
    height = None
    width = None
    if pd.notnull(description):
        dimensions = re.search(
            r"(?P<height>\d+)\s*(cm)?\s*x\s*(?P<width>\d+)\s*(cm)?", description, re.I
        )
        try:
            height = int(dimensions.group("height"))
            width = int(dimensions.group("width"))
        except AttributeError:
            pass
    return pd.Series([height, width])


def create_box(lat, lng, distance, bearings=[225, 45]):
    """
    See https://stackoverflow.com/a/76910048
    Uses centre point, distance, and bearing to get the corners of a bounding box.
    """
    l = []
    try:
        origin = geopy.Point(lat, lng)
    except ValueError:
        print(lat, lng)
    else:
        for bearing in bearings:
            destination = geodesic(miles=distance).destination(origin, bearing)
            coords = destination.longitude, destination.latitude
            l.extend(coords)
        # e, s, w, n
    return l


def get_diag_distance(height, width, scale):
    """
    Gets the distance from the centre of a map to a corner and uses the scale
    to convert the distance to miles.
    """
    d = math.sqrt(((height / 2) ** 2) + ((width / 2) ** 2))
    d_miles = ((d / 2.54) * scale) / 63360
    return d_miles


def get_bearings(height, width):
    """
    Get the angle to a corner of a map using its height and width.
    """
    bearing = math.degrees(math.atan((width / 2) / (height / 2)))
    return [bearing + 180, bearing]


def get_bbox(row):
    """
    Uses the available metadata to create a bounding box.
    """
    if (
        pd.isnull(row["height"])
        or pd.isnull(row["ratio"])
        or pd.isnull(row["latitude"])
        or pd.isnull(row["longitude"])
    ):
        return None
    distance = get_diag_distance(row["height"], row["width"], row["ratio"])
    bearings = get_bearings(row["height"], row["width"])
    bbox = create_box(row["latitude"], row["longitude"], distance, bearings)
    return bbox

In [7]:
df["image_id"] = df["alma_id"].progress_apply(get_image_id)

  0%|          | 0/8804 [00:00<?, ?it/s]

In [8]:
df[["latitude", "longitude"]] = df.progress_apply(get_coords, axis=1)

  0%|          | 0/8804 [00:00<?, ?it/s]

E1425399
S0378900
S0368500
E4733000
S0145590
E1418100
E1447600
S0378900
E1448400
S0377900
E1452190
(E 142°08ʹ--E 142°11ʹ/S 37°47ʹ--S 37°49ʹ).
E3745000
E1425399
S0363580
S0357000
E1448000
S0145590
E1418100
E1418100
S0363580
E4733000


In [9]:
df[["height", "width"]] = df["description"].progress_apply(get_size)

  0%|          | 0/8804 [00:00<?, ?it/s]

In [10]:
df[["scale", "ratio"]] = df["alma_id"].progress_apply(get_scale)

  0%|          | 0/8804 [00:00<?, ?it/s]

In [11]:
df["bbox"] = df.progress_apply(get_bbox, axis=1)

  0%|          | 0/8804 [00:00<?, ?it/s]

143.58333333333334 143.58333333333334
141.8 141.8


In [12]:
# This currently fails, some ids are added but eventually the manifest requests start generating 400 errors – not sure why
#df["iiif_ids"] = df["image_id"].progress_apply(get_iiif_ids)

In [17]:
df.head()

,alma_id,title,coordinates,east,south,publisher,description,image_id,latitude,longitude,height,width,scale,ratio,bbox
0,9921177713607636,"Paaratte, County of Heytesbury",(E 142°59'/S 38°33').,E1425900,S0383300,"Melbourne :Dept. of Lands and Survey,1882.",1 map ;on sheet 58 x 42 cm.,IE7030338,-38.550000,142.983333,58.0,42.0,"Scale [ca. 1:31,680]",31680.0,"[142.90692800485053, -38.63273691139862, 143.0..."
1,9921177763607636,"Powlett, County of Gladstone",(E 143°51'/S 36°25').,E1435100,S0362500,"Melbourne :Dept. of Lands and Survey,1883.",1 map ;on sheet 33 x 69 cm.,IE7041050,-36.416667,143.850000,33.0,69.0,"Scale [ca. 1:31,680]",31680.0,"[143.72806259777028, -36.46371024342854, 143.9..."
2,9921177843607636,"Patho, County of Gunbower",(E 144°22'/S 35°59').,E1442200,S0353900,"Melbourne :H. J. Green, Govt. Printer,1926.",1 map ;on sheet 54 x 81 cm.,IE7022188,-35.650000,144.366667,54.0,81.0,Scale [ca. 1:31 680],31680.0,"[144.2248526147727, -35.72700847677786, 144.50..."
3,9921177883607636,[Map of Parish of Stewarton],(E 145°50'/S 36°25' ).,E1455000,S0362500,"[Melbourne? :Dept. of Lands and Survey?,188-?].",1 map ;on sheet 30 x 41 cm.,IE7029775,-36.416667,145.833333,30.0,41.0,Scale not given,NaN,None
4,9921178363607636,Plan of a road from Stanley to Hillsborough an...,NaN,NaN,NaN,"Melbourne :Dept. of Lands and Survey,1867.",1 map ;on sheet 56 x 38 cm.,IE7041003,NaN,NaN,56.0,38.0,"Scale [ca. 1:15,840]",15840.0,None


In [19]:
total = df.shape[0]

Let's summarise the results:

In [28]:
has_coords = df.loc[(df["latitude"].notnull()) & (df["latitude"].notnull())].shape[0]
print(f"{has_coords/total:.02%} have coordinates")

38.06% have coordinates


In [29]:
has_size = df.loc[(df["height"].notnull()) & (df["width"].notnull())].shape[0]
print(f"{has_size/total:.02%} have dimensions")

50.43% have dimensions


In [30]:
has_ratio = df.loc[(df["ratio"].notnull())].shape[0]
print(f"{has_ratio/total:.02%} have scale")

55.79% have scale


In [31]:
has_bbox = df.loc[df["bbox"].notnull()].shape[0]
print(f"{has_bbox/total:.02%} have bounding boxes")

37.39% have bounding boxes


In [31]:
# Save to CSV
df.to_csv("parish_maps_final.csv", index=False)